In [1]:
# Load in packages for pandas, astropy, etc. 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 
import json 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)


In [7]:
# here we're working on format checks for the DR0.9 for the duals, binaries, and recoils.
# N-tuple tables are worked on below...
MAC = pd.read_csv('MAC_DR0p9.csv', sep=',')

MAC = MAC.drop(['Processed Selection Method','Analysis Method','Processed Analysis Method',\
                 'Processed Confirmation Method','Processed System Type','Confidence Flag','Selection Method',\
                 'Confirmation Method','Unnamed: 0','Unnamed: 0.1','System Type'], axis=1)

# dropping also the brightness measure columns in the DR0.9 but we will work to include this for the DR1 \
# at some point within the next month

MAC = MAC.drop(['Brightness1','Brightness_band1','Brightness_type1','Brightness2',\
                'Brightness_band2','Brightness_type2','Notes'], axis=1)
# also dropped my notes column for now

# dropping the original dV and dV_rwp columns....
# but note, there were discrepant values for dV (for example, comparing some of Liu's with my values) and we still \
# need to figure out why...
MAC = MAC.drop(['dV','dV_rwp','Legacy System Type'], axis=1)
# we're also dropping legacy system type for now because I need to go through and verify that corrections I \
# made previously to the individual nomenclature was preserved (for ex.: I converted 'binary quasar' \
# over to 'dual AGN' in my tables)
# we're also dropping the literature name for now. We need to add a query to simbad for something for the \
# vast majority of targets.... and we'll need to figure out how to handle cases where we have two nuclei that \
# have their own literature name...

# now reorganizing the columns and renaming things....

MAC.rename(columns={"dV_new": "dV", "Parsed Selection Method": "Selection Method",\
                   "Parsed Analysis Method": "Analysis Method",\
                   "Parsed Confirmation Method": "Confirmation Method"}, inplace=True)

#MAC

In [8]:
MAC.columns

Index(['Literature Name', 'Name1', 'z1', 'z1_type', 'RA1', 'Dec1',
       'Coordinate_waveband1', 'Coordinate_Source1', 'Equinox1', 'Name2', 'z2',
       'z2_type', 'RA2', 'Dec2', 'Equinox2', 'Coordinate_waveband2',
       'Coordinate_Source2', 'Sep', 'Sep(kpc)', 'Paper(s)', 'BibCode(s)',
       'DOI(s)', 'System Type 1: Dual AGNs', 'System Type 2: Binary AGNs',
       'System Type 3: Recoiling AGNs', 'ST1 Num AGNs', 'ST2 Num AGNs',
       'ST3 Num AGNs', 'ST1 Confidence Flag', 'ST2 Confidence Flag',
       'ST3 Confidence Flag', 'dV', 'Selection Method', 'Primary System Type',
       'Analysis Method', 'Confirmation Method'],
      dtype='object')

In [10]:
MAC = MAC[['System Type 1: Dual AGNs','ST1 Confidence Flag', 'System Type 2: Binary AGNs', 'ST2 Confidence Flag',\
       'System Type 3: Recoiling AGNs','ST3 Confidence Flag','Selection Method', 'Analysis Method',\
       'Confirmation Method','Name1', 'z1', 'z1_type', 'RA1', 'Dec1', \
       'Coordinate_waveband1','Coordinate_Source1', 'Equinox1', 'Name2', 'z2', 'z2_type', 'RA2','Dec2',\
       'Equinox2', 'Coordinate_waveband2', 'Coordinate_Source2', 'Sep','Sep(kpc)','dV',\
       'Paper(s)', 'BibCode(s)','DOI(s)']]


In [11]:
# Update on 16 July 2025
# Fixing redshifts for J0849+1114 and Arp 299

MAC.loc[MAC.Name1=="Arp 299", "z1"] = 0.01046
MAC.loc[MAC.Name1=="Arp 299", "z2"] =  0.010944
MAC.loc[MAC.Name1=="Arp 299", "Sep(kpc)"] =  4.6
# and the separation is 4.6 kpc


In [ ]:
#MAC.columns

In [ ]:
# based on the java table I just output for the duals, binaries, and recoils, I see I have pleny of updates \
# that need to be rolled out. 
# there are recoils without confidence flags, objects with no spec/phot flag for redshifts, objects flagged as 
# rank=1 but have separations that are way too large (object from Koss+2012). 
# We CAN make exceptions to the definitions, but we need to make specific notes for those somewhere, perhaps \
# on the website. 

# just double checked Mike's 2012 table. It lists NGC 4220 as the companion, but that the separation is only ~80 kpc
# which simply cannot be true. We'll need to double check. Maybe he meant it was the close companion to M106.


In [12]:
# now we're going to load in te multi-AGN tables, concatenate, and format!

#MACtriples.to_csv('MAC_tripleAGNs.csv', sep=',', index=False)
#MACquads.to_csv('MAC_quadrupleAGNs.csv', sep=',', index=False)
#liu2011quint.to_csv('MAC_quintupleAGN.csv', sep=',', index=False)
## note, the sextuplet will be listed in the catalog and online, but as an 'unlikely' system as far as \
## interaction driven. Hence, why the file name has 'bad' abended to it
#litsix.to_csv('MAC_sextupleAGN_bad.csv', sep=',', index=False)


# triples first
cairo = pd.read_csv('MAC_tripleAGNs.csv', sep=',')

# now quadruples
athens = pd.read_csv('MAC_quadrupleAGNs.csv', sep=',')

# now quintuples
malta = pd.read_csv('MAC_quintupleAGN.csv', sep=',')

# now sextuples


In [ ]:
# adding in missing paper, bibcode, and doi info:
amber = ['J075925.27+375216.0','J082846.95+122724.1','J085927.64+173344.9','J090515.73+075618.9',\
         'J095942.61+261914.2','J100722.96+230234.1','J102207.11+175017.2','J102223.90+154443.1',\
         'J102411.98+025234.2','J102546.26+134300.7','J105609.41+465341.7','J110441.92+041750.4',\
         'J110517.99+005328.3','J110719.72+025820.5','J111005.18+032833.7','J114938.17-033408.7',\
         'J120550.07+202837.2','J120718.36+204450.9','J121425.18+082251.9','J121442.50+125924.8',\
         'J131628.81+525737.2','J132356.94+305917.0','J133227.93+071859.2','J141057.24+252950.0',\
         'J141151.61-000303.2','J141421.15+025721.2','J143430.61+230529.9','J143801.02+101051.5',\
         'J150631.41+404146.4','J150821.57+191131.6','J151310.42+042855.5','J151640.00+215938.7',\
         'J160633.13+174632.3','J160709.88+183828.6','J162906.60+114231.2','J163300.85+114510.1',\
         'J231138.88+150741.7','J231224.52+135704.6','J232402.98-095341.9']

for index, row in cairo.iterrows():
    if row['Name1'] in amber:
        cairo.at[index, 'Paper(s)'] = "Liu+2011b"
        cairo.at[index, 'BibCode(s)'] = "2011ApJ...737..101L"
        cairo.at[index, 'DOI(s)'] = "https://doi.org/10.1088/0004-637X/737/2/101"

cairo = cairo[['Primary System Type','ST1 Confidence Flag', 'Secondary System Type','ST2 Confidence Flag', \
    'Tertiary System Type', 'ST3 Confidence Flag','Selection Method', 'Confirmation Method','Literature Name',\
    'Name1', 'RA1', 'Dec1', 'Name2', 'RA2', 'Dec2', 'Name3', 'RA3', 'Dec3','z1', 'z1_type', 'Equinox1',\
    'Coordinate_waveband1', 'Coordinate_Source1', 'AGN_type1', 'z2','z2_type', 'Equinox2', \
    'Coordinate_waveband2', 'Coordinate_Source2', 'AGN_type2', 'z3', 'z3_type', 'Equinox3', \
    'Coordinate_waveband3','Coordinate_Source3', 'AGN_type3', 'Sep23', 'Sep12', 'Sep13','Sep(kpc)12_z1',\
    'Sep(kpc)12_z2', 'Sep(kpc)13_z1', 'Sep(kpc)13_z3','Sep(kpc)23_z2', 'Sep(kpc)23_z3', 'dV_12', 'dV_13', 'dV_23',\
    'Paper(s)', 'BibCode(s)','DOI(s)']]

# droppig these columns for now...
#'ST3 Confidence Rank','Brightness1','Brightness_band1', 'Brightness_type1', 'Brightness2',
#'Brightness_band2', 'Brightness_type2', 'Brightness3','Brightness_band3', 'Brightness_type3',
#'Notes'

## fixing the redshifts for J0849+1114 now 
#cairo.loc[cairo.Name1=="", "z1"] = 
#cairo.loc[cairo.Name1=="", "z2"] = 
#cairo.loc[cairo.Name1=="", "z3"] = 


# adding in missing paper, bibcode, and doi info:
clad = ['J135632.71+232118.3','J144702.08+134006.2']
for index, row in athens.iterrows():
    if row['Name1'] in clad:
        athens.at[index, 'Paper(s)'] = "Liu+2011b"
        athens.at[index, 'BibCode(s)'] = "2011ApJ...737..101L"
        athens.at[index, 'DOI(s)'] = "https://doi.org/10.1088/0004-637X/737/2/101"

athens = athens[['Primary System Type', 'ST1 Confidence Flag', 'Secondary System Type', 'ST2 Confidence Flag',\
                 'Tertiary System Type','ST3 Confidence Flag','Literature Name',\
                 'Selection Method','Confirmation Method','Name1','z1','z1_type','AGN_type1','RA1','Dec1','Equinox1',\
                 'Coordinate_waveband1', 'Coordinate_Source1','Name2','z2', 'z2_type','AGN_type2','RA2','Dec2',\
                 'Equinox2', 'Coordinate_waveband2','Coordinate_Source2','Name3','AGN_type3','z3',\
                 'z3_type','RA3', 'Dec3','Equinox3','Coordinate_waveband3','Coordinate_Source3',\
                 'Name4', 'z4','z4_type','AGN_type4','RA4', 'Dec4','Equinox4','Coordinate_waveband4',\
                 'Coordinate_Source4','Sep12', 'Sep13', 'Sep14', 'Sep23', 'Sep24', 'Sep34','Sep(kpc)12_z1',\
                 'Sep(kpc)12_z2', 'Sep(kpc)13_z1', 'Sep(kpc)13_z3','Sep(kpc)23_z2', 'Sep(kpc)23_z3',\
                 'Sep(kpc)14_z1', 'Sep(kpc)14_z4','Sep(kpc)24_z2', 'Sep(kpc)24_z3', 'Sep(kpc)34_z3',\
                 'Sep(kpc)34_z4','dV_12', 'dV_13', 'dV_14', 'dV_23', 'dV_24', 'dV_34','Paper(s)',\
                 'BibCode(s)', 'DOI(s)']]
       

# dropping these for now...
#'Brightness1', 'Brightness_band1',
#'Brightness_type1', 'Brightness2', 'Brightness_band2',
#'Brightness_type2', 'Brightness3', 'Brightness_band3',
#'Brightness_type3', 'Brightness4', 'Brightness_band4',
#'Brightness_type4', 
#'Notes',

# adding in this information here since it was lost in the N-tuple formattig notebook
# we need to go back and update the N-tuple notebook to add these kinds of revisions in there
malta['Literature Name'] = "SDSS J0959+1259 (CG)"
malta['DOI(s)'] = "2011ApJ...737..101L ; 2015MNRAS.453..214D"
malta['Paper(s)'] = "Liu+2011b ; De Rosa+2015"
malta['BibCode(s)'] = "https://doi.org/10.1088/0004-637X/737/2/101 ; https://doi.org/10.1093/mnras/stv1623"
malta['Notes'] = "The separations quoted here are for the distance of each source to the ***center*** of the compact group. This system is composed of 2 Sy2s 1 Sy1 and 2 LINERs."


malta = malta[['Primary System Type','ST1 Confidence Flag','Secondary System Type', 'ST2 Confidence Flag',\
               'Tertiary System Type','ST3 Confidence Flag', 'Literature Name',\
               'Selection Method','Confirmation Method',\
               'Name1', 'RA1', 'Dec1', 'z1', 'z1_type', 'Equinox1','Coordinate_waveband1',\
               'Coordinate_Source1', 'AGN_type1', 'Name2', 'RA2', 'Dec2', 'z2','z2_type', 'Equinox2',\
               'Coordinate_waveband2', 'Coordinate_Source2','AGN_type2', 'Name3', 'RA3', 'Dec3', 'z3',\
               'z3_type', 'Equinox3', 'Coordinate_waveband3','Coordinate_Source3', 'AGN_type3', 'Name4',\
               'RA4', 'Dec4', 'z4', 'z4_type', 'Equinox4','Coordinate_waveband4', 'Coordinate_Source4',\
               'AGN_type4', 'Name5', 'RA5', 'Dec5','z5','z5_type', 'Equinox5', 'Coordinate_waveband5',\
               'Coordinate_Source5','AGN_type5', 'Sep12', 'Sep13', 'Sep14', 'Sep15', 'Sep23', 'Sep24',\
               'Sep25', 'Sep34', 'Sep35', 'Sep45', 'Sep(kpc)12_z1', 'Sep(kpc)12_z2','Sep(kpc)13_z1',\
               'Sep(kpc)13_z3', 'Sep(kpc)14_z1', 'Sep(kpc)14_z4','Sep(kpc)15_z1', 'Sep(kpc)15_z5',\
               'Sep(kpc)23_z2', 'Sep(kpc)23_z3','Sep(kpc)24_z2', 'Sep(kpc)24_z4', 'Sep(kpc)25_z2',\
               'Sep(kpc)25_z5','Sep(kpc)34_z3', 'Sep(kpc)34_z4', 'Sep(kpc)35_z3', 'Sep(kpc)35_z5',\
               'Sep(kpc)45_z4', 'Sep(kpc)45_z5', 'dV_12', 'dV_13', 'dV_14', 'dV_15','dV_23', 'dV_24',\
               'dV_25', 'dV_34', 'dV_35', 'dV_45','Paper(s)','BibCode(s)','DOI(s)']]


In [16]:
#cairo

In [17]:
# saving the .csv files here now for the DR1 version (of course, plenty of updates are still needed)

MAC.to_csv("DR1_tabledownloads/BigMAC_maintable_DR1.csv", sep=',', index=False)
cairo.to_csv("DR1_tabledownloads/BigMAC_tripletable_DR1.csv", sep=',', index=False)
athens.to_csv("DR1_tabledownloads/BigMAC_quadtable_DR1.csv", sep=',', index=False)
malta.to_csv("DR1_tabledownloads/BigMAC_quinttable_DR1.csv", sep=',', index=False)


In [19]:
# Generating Java script tables here for the website based on DR0.9 for now:

from astropy.table import Table

t = Table.from_pandas(MAC)
t.write('MACmain_DR1_webtable_18July2025.html', format='jsviewer', overwrite=True)

t = Table.from_pandas(cairo)
t.write('MACtriples_DR1_webtable_18July2025.html', format='jsviewer', overwrite=True)

t = Table.from_pandas(athens)
t.write('MACquads_DR1_webtable_18July2025.html', format='jsviewer', overwrite=True)

t = Table.from_pandas(malta)
t.write('MACquints_DR1_webtable_18July2025.html', format='jsviewer', overwrite=True)



In [ ]:
#MACmain_DR0p9_webtable_20Nov2024.html
#MACtriples_DR0p9_webtable_20Nov2024.html
#MACquads_DR0p9_webtable_20Nov2024.html
#MACquints_DR0p9_webtable_20Nov2024.html